In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split

# unpack the pickle
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset_dir = '/content/drive/MyDrive/FALL_2022_SEMESTER/COMS_4995_Applied_Deep_Learning/Course_Project/music_dataset/'
pickle_file_path = os.path.join(dataset_dir, 'first_run_pic_1.pickle')

In [3]:
# unpack the pickle file
pickle_in = open(pickle_file_path, "rb")
test_data = pickle.load(pickle_in)

In [4]:
# map each raga str to an int
raga_to_int = {
    'Sankarabaranam': 0,
    'Reethigowla': 1,
    'Kalyani': 2,
    'Shanmukhapriya': 3,
    'SindhuBhairavi': 4,
    'Sri': 5,
    'Surutti': 6,
    'Thodi': 7,
    'PoorviKalyani': 8,
    'Mohanam': 9,
    'Madhyamavathi': 10,
    'Kharaharapriya': 11,
    'Keeravani': 12,
    'Kapi': 13,
    'Kambhodhi': 14,
    'Kamas': 15, 
    'Jaganmohini': 16,
    'Hamsadhwani': 17,
    'Bilahari': 18,
    'Bhairavi': 19
}


In [5]:
# OK so now just create a random network and see how it does
# split up into test and train data
X = []
y = []
for val in test_data:
  X.append(val[0])
  y.append(raga_to_int[val[1]])

del test_data




In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)
X_train =np.array(X_train)
X_test =np.array(X_test)
y_train =np.array(y_train)
y_test =np.array(y_test)

In [7]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (1173, 1323000)
X_test shape: (391, 1323000)
y_train shape: (1173,)
y_test shape: (391,)


In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1323000, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(20)
])

In [9]:
predictions = model(X_train[:1]).numpy()
predictions

array([[-3295.6458 , -9772.099  ,  7145.149  ,  -465.78326,   503.1726 ,
         6078.     ,  9639.982  , -1892.6475 ,  5445.39   ,  1265.1707 ,
        -3540.7725 ,  4865.011  ,  3934.079  , -6354.7427 ,  2069.5564 ,
         4367.538  ,  6750.834  , -2480.6394 ,  3694.4136 , -2368.8464 ]],
      dtype=float32)

In [10]:
tf.nn.softmax(predictions).numpy()


array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=1)


37/37 [==============================] - 46s 1s/step - loss: 4431.8110 - accuracy: 0.9685


In [16]:
# might have started to overfit on trarin accuracy after 1 epoch???
# try testing it 
model.evaluate(X_test,  y_test, verbose=2)


13/13 - 5s - loss: 290441.2812 - accuracy: 0.1611 - 5s/epoch - 378ms/step


[290441.28125, 0.1611253172159195]

In [ ]:
# ok so now, get the actual features of the dataset (Spectral centroid, spectral bandwidth, mel-spectrogram, Mel freq cepstral coeff, chroma)
# Get those into a pickle file, and see how it performs on the same neural network. If accuracy is low, then either modify network or add more data samples